In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile

zip_path = '/content/drive/MyDrive/cat_dog_data.zip'  # change if inside a folder
extract_to = '/content/cat_dog_dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [4]:
!ls /content/cat_dog_dataset/PetImages

Cat  Dog


In [5]:
import os, shutil, random

base_dir = '/content/cat_dog_dataset'
src_dir = os.path.join(base_dir, 'PetImages')

# Create target directories
sets = ['train', 'validation', 'test']
categories = ['cats', 'dogs']

for s in sets:
    for cat in categories:
        os.makedirs(os.path.join(base_dir, s, cat), exist_ok=True)

def split_data(category, label):
    src_folder = os.path.join(src_dir, category)
    images = [img for img in os.listdir(src_folder) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    n = len(images)
    train_split = int(0.7 * n)
    val_split = int(0.15 * n)

    train_imgs = images[:train_split]
    val_imgs = images[train_split:train_split + val_split]
    test_imgs = images[train_split + val_split:]

    for img in train_imgs:
        try:
            shutil.copy(os.path.join(src_folder, img), os.path.join(base_dir, 'train', label, img))
        except:
            pass

    for img in val_imgs:
        try:
            shutil.copy(os.path.join(src_folder, img), os.path.join(base_dir, 'validation', label, img))
        except:
            pass

    for img in test_imgs:
        try:
            shutil.copy(os.path.join(src_folder, img), os.path.join(base_dir, 'test', label, img))
        except:
            pass

# Run for both categories
split_data('Cat', 'cats')
split_data('Dog', 'dogs')

In [6]:
train_dir = '/content/cat_dog_dataset/train'
val_dir = '/content/cat_dog_dataset/validation'
test_dir = '/content/cat_dog_dataset/test'

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen= ImageDataGenerator(rescale=1./255)

In [9]:
train_gen= train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    batch_size= 32,
    class_mode= "binary"
)

valid_gen= val_test_datagen.flow_from_directory(
    val_dir,
    target_size= (150,150),
    batch_size= 32,
    class_mode= "binary"
)

test_gen= val_test_datagen.flow_from_directory(
    test_dir,
    target_size= (150,150),
    batch_size= 32,
    class_mode= "binary",
    shuffle= False
)

Found 17498 images belonging to 2 classes.
Found 3748 images belonging to 2 classes.
Found 3752 images belonging to 2 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),  # Extract basic features (edges, corners)
    MaxPooling2D(2, 2),  # Downsample to reduce spatial size and computation

    Conv2D(64, (3,3), activation='relu'),  # Extract deeper/more complex features
    MaxPooling2D(2, 2),  # Further downsampling

    Conv2D(128, (3,3), activation='relu'),  # Learn even higher-level features
    MaxPooling2D(2, 2),  # Reduce size while keeping important features

    Flatten(),  # Convert 3D feature maps to 1D vector for dense layers

    Dense(512, activation='relu'),  # Fully connected layer to learn non-linear combinations
    Dropout(0.5),  # Prevent overfitting by randomly deactivating 50% neurons

    Dense(1, activation='sigmoid')  # Output layer for binary classification (cat or dog)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [12]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=valid_gen
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 1193s 2s/step - accuracy: 0.5236 - loss: 0.7098 - val_accuracy: 0.5414 - val_loss: 0.6872
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 1200s 2s/step - accuracy: 0.5589 - loss: 0.6800 - val_accuracy: 0.6070 - val_loss: 0.6448
Epoch 3/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 1156s 2s/step - accuracy: 0.6080 - loss: 0.6536 - val_accuracy: 0.6966 - val_loss: 0.5939
Epoch 4/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 1164s 2s/step - accuracy: 0.6410 - loss: 0.6309 - val_accuracy: 0.7038 - val_loss: 0.5683
Epoch 5/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 1165s 2s/step - accuracy: 0.6867 - loss: 0.5861 - val_accuracy: 0.7124 - val_loss: 0.5557
Epoch 6/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 1166s 2s/step - accuracy: 0.7091 - loss: 0.5601 - val_accuracy: 0.7372 - val_loss: 0.5458
Epoch 7/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 1167s 2s/step - accuracy: 0.7248 - loss: 0.5428 - val_accuracy: 0.7559 - val_loss: 0.5015
Epoch 8/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 1222s 2s/step - accuracy: 0.7371 - loss: 0.5275 - 

In [13]:
model.save('/content/drive/MyDrive/cat_dog_model.h5')

In [14]:
history.history['accuracy']

[0.5457195043563843,
 0.5723510980606079,
 0.6193850636482239,
 0.6546462178230286,
 0.6876785755157471,
 0.7124242782592773,
 0.7205395102500916,
 0.7393416166305542,
 0.7461995482444763,
 0.7551148533821106]

In [15]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc:.4f}")

111/118 ━━━━━━━━━━━━━━━━━━━━ 3s 519ms/step - accuracy: 0.7632 - loss: 0.5075

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


118/118 ━━━━━━━━━━━━━━━━━━━━ 61s 512ms/step - accuracy: 0.7655 - loss: 0.5035
Test Accuracy: 0.7974


In [16]:
print(test_gen.class_indices)

{'cats': 0, 'dogs': 1}


In [17]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get predictions
test_probs = model.predict(test_gen)
test_preds = (test_probs > 0.5).astype(int).reshape(-1)
true_labels = test_gen.classes
class_names = list(test_gen.class_indices.keys())

# Metrics
print(f"✅ Test Accuracy: {test_acc:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, test_preds))
print("\nClassification Report:")
print(classification_report(true_labels, test_preds, target_names=class_names))

118/118 ━━━━━━━━━━━━━━━━━━━━ 61s 505ms/step
✅ Test Accuracy: 0.7974

Confusion Matrix:
[[1409  467]
 [ 293 1583]]

Classification Report:
              precision    recall  f1-score   support

        cats       0.83      0.75      0.79      1876
        dogs       0.77      0.84      0.81      1876

    accuracy                           0.80      3752
   macro avg       0.80      0.80      0.80      3752
weighted avg       0.80      0.80      0.80      3752

